In [1]:
import numpy as np
import string
import re
import requests
np.random.seed(1234)

In [2]:
initial={}
first_order={}
second_order={}

In [3]:
def remove_punctuation(text):
    return re.sub('[%s]' % re.escape(string.punctuation), '', text)

In [4]:
alan=requests.get('https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/edgar_allan_poe.txt')
alan=alan.text

In [5]:
def add2dict(dict_, k, v):
    if k not in dict_:
        dict_[k]=[]
    dict_[k].append(v)


In [6]:
for line in alan.split('\n'):
    tokens=remove_punctuation(line.rstrip().lower()).split()
    T=len(tokens)
    for i in range(len(tokens)):
        t=tokens[i]
        if i==0:
            initial[t]=initial.get(t, 0.)+1
        else:
            t_1=tokens[i-1]
            if i==1:
                add2dict(first_order, t_1, t)
            if i==T-1:
                add2dict(second_order, (t_1, t), 'END')
            else:
                t_2=tokens[i-2]
                add2dict(second_order, (t_2, t_1), t)

In [7]:
len(second_order)

2734

In [8]:
initial_total=sum(initial.values())
for k, v in initial.items():
    initial[k]=v/initial_total
# initial

In [9]:
def list2prob(li):
    count={}
    for word in li:
        count[word]=count.get(word, 0)+1
    n = len(li)
    for k, v in count.items():
        count[k]=v/n
    return count

In [10]:
for k, v in first_order.items():
    first_order[k]=list2prob(v)
# first_order_prob

In [11]:
for k, v in second_order.items():
    second_order[k]=list2prob(v)
# second_order_prob

In [12]:
def sample_word(d):
    p=np.random.random()
    cummulative_prob=0
    for k, v in d.items():
        cummulative_prob+=v
        if p<cummulative_prob:
            return k
    print(d.values())
    assert False

In [13]:
  
def generate_word():
    for _ in range(4):
        sentence=[]
        w0 = sample_word(initial)
        sentence.append(w0)
        w1=sample_word(first_order[w0])
        sentence.append(w1)
        while True:
            while (w0, w1) not in second_order:
                sentence=[]
                w0 = sample_word(initial)
                sentence.append(w0)
                w1=sample_word(first_order[w0])
                sentence.append(w1)
            w2=sample_word(second_order[(w0, w1)])
            if w2=='END':
                break
            sentence.append(w2)
            w0=w1
            w1=w2
        print(' '.join(sentence))

In [14]:
generate_word()

no more
come up in despite of the moon
through which came flowing flowing
at the end of the bells
